<a href="https://colab.research.google.com/github/nike-2001/AI-Hands-on/blob/main/Copy_of_Applying_k_NN_to_MNIST_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

## Downloading MNIST Train and Test Datasets  
 
* **Proceed to further steps only after executing the cells in this section**.
* The variables from these steps are used in some of the sample test cases.

In [ ]:
# Downloading the datasets using wget
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/mnist_train.csv
!wget https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/mnist_test.csv

--2021-01-25 02:23:48--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/mnist_train.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.219.64.55
Connecting to nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)|52.219.64.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109575994 (104M) [text/csv]
Saving to: ‘mnist_train.csv’

mnist_train.csv     100%[===================>] 104.50M  12.1MB/s    in 10s     

2021-01-25 02:24:00 (10.1 MB/s) - ‘mnist_train.csv’ saved [109575994/109575994]

--2021-01-25 02:24:00--  https://nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com/otg_prod/media/Tech_4.0/AI_ML/Datasets/mnist_test.csv
Resolving nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com (nkb-backend-otg-media-static.s3.ap-south-1.amazonaws.com)... 52.

**NOTE:** Executing the below cell might take some time (1-2 min) as the original MNIST dataset is large.

In [ ]:
train_file_name = "mnist_train.csv"
train_data = np.genfromtxt(train_file_name, delimiter=',', dtype=np.uint16)
print(f"Shape of the train_data in {train_file_name} is: {train_data.shape} \n")

Shape of the train_data in mnist_train.csv is: (60000, 785) 



In [ ]:
MNIST_train_Y = train_data[:, 0].reshape(-1, 1)
MNIST_train_X = train_data[:, 1:]

print(f"Shape of X: {MNIST_train_X.shape} \n")
print(f"Shape of Y: {MNIST_train_Y.shape} \n")

Shape of X: (60000, 784) 

Shape of Y: (60000, 1) 



**NOTE:** We've used **`np.uint16`** to reduce the space taken by the input arrays.


In [ ]:
test_file_name = "mnist_test.csv"
test_data = np.genfromtxt(test_file_name, delimiter=',', dtype=np.uint16)
print(f"Shape of the test_data in {test_file_name} is: {test_data.shape} \n")

Shape of the test_data in mnist_test.csv is: (10000, 785) 



In [ ]:
MNIST_test_Y = test_data[:, 0].reshape(-1, 1)
MNIST_test_X = test_data[:, 1:]

print(f"Shape of X: {MNIST_test_Y.shape} \n")
print(f"Shape of Y: {MNIST_test_X.shape} \n")

Shape of X: (10000, 1) 

Shape of Y: (10000, 784) 



### Sizes of Train and Test datasets

In [ ]:
print(f"Size of train_X: {MNIST_train_X.nbytes}")
print(f"Size of train_Y: {MNIST_train_Y.nbytes}")

Size of train_X: 94080000
Size of train_Y: 120000


In [ ]:
print(f"Size of test_X: {MNIST_test_X.nbytes}")
print(f"Size of test_Y: {MNIST_test_Y.nbytes}")

Size of test_X: 15680000
Size of test_Y: 20000


## k-NN Algorithm

### Split Train and Validation Data

In [ ]:
import math
def shuffle(X, Y):
  np.random.seed(2) 
  indices = np.random.permutation(X.shape[0])
  shuffled_X = X[indices]
  shuffled_Y = Y[indices]
  return shuffled_X, shuffled_Y

Using a fixed validation set size instead of percentage



In [ ]:
  inputs, labels = shuffle(MNIST_train_X, MNIST_train_Y)
  train_length = 59000
  
  train_inputs = inputs[:train_length]
  train_labels = labels[:train_length]
  validation_inputs = inputs[train_length:]
  validation_labels = labels[train_length:]

In [ ]:
print(f"Size of train_inputs: {train_inputs.nbytes}")
print(f"Size of validation_inputs: {validation_inputs.nbytes}")

Size of train_inputs: 92512000
Size of validation_inputs: 1568000


### Compute distances matrix
We're computing the distances between all the validation inputs and training inputs beforehand, so that we need not compute them in every iteration of **`majority_based_knn`** function.

**NOTE:** We've used **`np.float32`** to reduce the space taken by the input arrays.

In [ ]:
train_count = train_inputs.shape[0]
num_of_features = train_inputs.shape[1]
validation_count = validation_inputs.shape[0]

distances_matrix = np.zeros((validation_count, train_count), dtype=np.float32)

In [ ]:
print(f"Size of distances_matrix: {distances_matrix.nbytes}")

Size of distances_matrix: 236000000


In [ ]:
def Ln_norm_distances(train_X, test_x, n):
    abs_diff = np.abs(train_X - test_x)  
    summation = np.sum(np.power(abs_diff, n), axis=1)
    ln_distances = np.power(summation, 1/n)
    return ln_distances

We are computing L2 norm distances

In [ ]:
n = 2
import time
for idx, each in enumerate(validation_inputs):
  ln_distances = Ln_norm_distances(train_inputs, each, n)
  distances_matrix[idx] = ln_distances

### Majority Based k-NN

We've updated the **`majority_based_knn`** function to use the `distances_matrix` which is computed beforehand.

In [ ]:
def majority_based_knn(distances_matrix, train_Y, k):
  unique_class_labels = np.unique(train_Y)
  num_unique_classes = unique_class_labels.shape[0]

  train_length = distances_matrix.shape[1]
  test_length = distances_matrix.shape[0]

  label_wise_counts = np.zeros((test_length, num_unique_classes))
  label_wise_weights = np.zeros((test_length, num_unique_classes))

  sorted_indices = np.argsort(distances_matrix, axis=1)

  for test_idx in range(test_length):
    # Getting k-Nearest Neighbors from distances matrix
    test_distances = distances_matrix[test_idx]
    sorted_test_indices = sorted_indices[test_idx]
    kth_dist_repeat_count = 0
    if train_length > k:
      kth_neighbour_distance = test_distances[sorted_test_indices[k-1]] 
      kth_dist_repeat_count = np.count_nonzero(test_distances[k:] == kth_neighbour_distance)
    indices_k = sorted_test_indices[:(k + kth_dist_repeat_count)]
    distances_k = test_distances[indices_k]
    labels_k = train_Y[indices_k]

    for label_idx, each_label in enumerate(unique_class_labels):
      label_weight = np.sum(np.where(labels_k == each_label, 1/distances_k, 0.0))
      label_wise_weights[test_idx][label_idx] = label_weight
      label_count = np.sum(np.where(labels_k == each_label, 1.0, 0.0))
      label_wise_counts[test_idx][label_idx] = label_count
  
  output_labels = np.empty(test_length, dtype=int)

  sorted_counts_indices = np.argsort(label_wise_counts, axis=1)
  for test_idx, test_indices in enumerate(sorted_counts_indices):
    highest_count = label_wise_counts[test_idx][test_indices[num_unique_classes-1]]
    highest_label_repeat = np.count_nonzero(label_wise_counts[test_idx] == highest_count)
    
    no_voting_tie = (highest_label_repeat==1)
    if no_voting_tie:
      output_labels[test_idx] = unique_class_labels[test_indices[num_unique_classes-1]]
    else:
      tied_class_indices = test_indices[num_unique_classes-highest_label_repeat:]
      tied_class_weights = label_wise_weights[test_idx][tied_class_indices]
      max_weight_idx = np.argmax(tied_class_weights)
      max_idx = tied_class_indices[max_weight_idx]
      output_labels[test_idx] = unique_class_labels[max_idx]

  return output_labels

In [ ]:
def calculate_accuracy(predicted_labels, actual_labels):
    correctly_predicted_count = np.count_nonzero(predicted_labels == actual_labels)
    accuracy = float(correctly_predicted_count)/predicted_labels.size
    return accuracy

**NOTE:** **`distances_matrix`** contains 'L2' distances between training and validation data

In [ ]:
k = 20
output_labels = majority_based_knn(distances_matrix, train_labels, k)
accuracy = calculate_accuracy(output_labels.flatten(), validation_labels.flatten())
print(f"accuracy for (k, n) {k , n} is : {accuracy}")

accuracy for (k, n) (20, 2) is : 0.966


### Best `(k, n)` pair

Compute the best `(k, n)` pair based on the above changes

In [ ]:
# ToDo